In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
import ConeccionesSucursales as cs 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 1

In [3]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [4]:
#Archivo que contiene los puestos de cada persona
book = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Personal/')
personal = pd.DataFrame()
for i in book:
    new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Personal/'+i+'')
    personal = pd.concat([personal,new],ignore_index=True)
personal = personal[['USUARIO','COBRANZA/CAPTURISTA']]
personal.columns = ['Usuario','Estatus']
personal.head()

,Usuario,Estatus
0,CFLORES,ADMIN
1,WGALICIA,ADMIN
2,AHERNANDEZS,ADMIN
3,CRSALINASDM,ADMIN
4,VALENCIAMICA,ADMIN


In [5]:
#Lectura pagos Banco Azteca
sheet_baz_renta = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta')
pagos_baz_renta = pd.DataFrame()
for i in sheet_baz_renta:
    pagos_baz_renta_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_baz_renta_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta2 = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta2 = pd.DataFrame(pagos_baz_renta2.to_records())

In [6]:
#Lectura pagos Banco Azteca
sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for k in sheet_baz_recovery:
    pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
pagos_baz_recovery['Cartera'] = 'Recovery'
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [7]:
#Lectura pagos Banco Azteca
sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for j in sheet_baz_26:
    pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
pagos_baz_26['cartera'] = '26-39'
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()
pagos_baz_26.head(1)

,ANIO,SEMANA,DIA,PAIS,CANAL,SUCURSAL,FOLIO,RECUPERACION DE CAPITAL,RECUPERACION DE MORATORIOS,SALDO ACTUAL,MORATORIOS ACTUAL,FECHA GESTION,CARGO AUTOMATICO,cartera,credito
0,2021,2,2021-01-11,1,1,2161,90848,0,350,12273.0,3056.0,11/01/2021,NO,26-39,1-1-2161-90848


In [8]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Llamadas Detallado Banco Azteca

In [9]:
#Hacemos la consulta referente a las gestiones de Credifiel
#llamadas_cred = consulta_sql('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^BAZ)')
#llamadas_cred.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id',#'list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
#llamadas_cred.head(1)

In [10]:
#llamadas_cred['turno'] = np.where(llamadas_cred['call_time_hour']<=14,'M','V')

# Performance Banco Azteca

In [11]:
#Hacemos la consulta referente al performance del Gestor
performance_all = cs.query_naucalpan('procesos_externos.all_rpt_agent_performance',mes,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [12]:
performance_all['user_group2'] = [x[:3] for x in performance_all['user_group']]
performance_baz = performance_all.loc[performance_all['user_group2']=='BAZ']
performance_baz[['ocupacion','adherencia']] = percent_tonumeric(performance_baz,['ocupacion','adherencia'])

In [13]:
performance_baz['hora_login'] = [int(x[:2]) for x in performance_baz['login_vici']]
performance_baz['hora_logout'] = [int(x[:2]) for x in performance_baz['logout_vici']]
performance_baz['turno'] = np.where((performance_baz['hora_logout']<=14)&(performance_baz['hora_logout']-performance_baz['hora_login']<=8),'M',
                                    np.where((performance_baz['hora_logout']>17)&(performance_baz['hora_logout']-performance_baz['hora_login']<=8),'V','Mixto'))

In [14]:
performance_baz['horas_trabajadas'] = performance_baz['hora_logout']-performance_baz['hora_login']
performance_baz['llamadas_hora'] = performance_baz['llamadas']/performance_baz['horas_trabajadas']
performance_baz['llamadas_hora'] = performance_baz['llamadas_hora'].round(1)

In [15]:
control_performance_baz_turno = pd.pivot_table(performance_baz,index=['usuario','nombre_usuario','turno'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_baz_turno = to_dataframe(control_performance_baz_turno)
control_performance_baz_turno = control_performance_baz_turno.iloc[:,[0,1,2,3,4,5,6,7,8,9,12,13,14,15,16,17]]
control_performance_baz_turno.columns = ['Usuario','Gestor','Turno','SumHorasTrabajadas','SumLlamadasDia','SumOcupacion','PromedioHorasTrab','PromedioLlamadasDia','PromedioOcupacion','DiasAsitencia','MinHoras','MinLlamadasDia','MinOcupacion','MaxHoras','MaxLlamadasDia','MaxOcupacion']
control_performance_baz_turno['LlamadasHora'] = control_performance_baz_turno['SumLlamadasDia']/control_performance_baz_turno['SumHorasTrabajadas']
control_performance_baz_turno.head(2)

,Usuario,Gestor,Turno,SumHorasTrabajadas,SumLlamadasDia,SumOcupacion,PromedioHorasTrab,PromedioLlamadasDia,PromedioOcupacion,DiasAsitencia,MinHoras,MinLlamadasDia,MinOcupacion,MaxHoras,MaxLlamadasDia,MaxOcupacion,LlamadasHora
0,AGUILARRIAL,AGUILAR RIOS DANIEL ALEJANDRO,M,6,154,11.75,6.000000,154.0,11.750000,1,6,154,11.75,6,154,11.75,25.666667
1,ALHENANDEZA,ADRIANA ALVARADO HERNANDEZ,M,20,621,90.16,6.666667,207.0,30.053333,3,6,186,26.41,7,219,35.60,31.050000


In [16]:
control_performance_baz_turno2 = control_performance_baz_turno[['Gestor','Turno','LlamadasHora','PromedioOcupacion']]

In [17]:
control_performance_baz = pd.pivot_table(performance_baz,index=['usuario'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_baz = to_dataframe(control_performance_baz)
control_performance_baz = control_performance_baz.iloc[:,[0,1,2,5,6]]
control_performance_baz.columns = ['Usuario','SumHorasTrabajadas','SumLlamadasDia','PromedioLlamadasDia','PromedioOcupacion']
control_performance_baz['PromedioLlamadasHora'] = control_performance_baz['SumLlamadasDia']/control_performance_baz['SumHorasTrabajadas']
cols = ['PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion']
control_performance_baz[cols] = control_performance_baz[cols].round()
control_performance_baz.head(3)

,Usuario,SumHorasTrabajadas,SumLlamadasDia,PromedioLlamadasDia,PromedioOcupacion,PromedioLlamadasHora
0,AGUILARRIAL,6,154,154.0,12.0,26.0
1,ALHENANDEZA,208,6267,190.0,29.0,30.0
2,ALVARADORODAAR,6,123,62.0,42.0,20.0


# Gestion Banco Azteca

In [18]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = cs.query_naucalpan('baz_rpt_gestiones_detallado',mes,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']

In [19]:
#try: gestion_baz['anio_prom'] = [int(x[:4]) for x in gestion_baz['fecha_promesa']]
#except: gestion_baz['anio_prom'] = 0
#gestion_baz = gestion_baz.loc[gestion_baz['anio_prom']<2030]
gestion_baz = gestion_baz.drop_duplicates()
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz['usuario'] = [x.rstrip() for x in gestion_baz['usuario']]
#gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)

In [20]:
gestion_gestor_rec = pd.pivot_table(gestion_baz,index=['usuario','nombre_usuario'],values=['fecha_gestion'],aggfunc='count')
gestion_gestor_rec = to_dataframe(gestion_gestor_rec)
gestion_gestor_rec.columns = ['Usuario','NombreUsuario','NumeroGestiones']

In [21]:
promesas_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
promesas_baz = promesas_baz.loc[(promesas_baz['monto_promesa']>10) & (promesas_baz['monto_promesa']<500000)]

In [22]:
pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
consolidado_pagos_baz = pd.merge(promesas_baz,pagos_baz,how='left',on=['credito'])
#consolidado_pagos_baz.head(1)

In [23]:
z = consolidado_pagos_baz.loc[consolidado_pagos_baz['RECUPERACION DE CAPITAL']+consolidado_pagos_baz['RECUPERACION DE MORATORIOS']>consolidado_pagos_baz['monto_promesa']]

In [24]:
consolidado_pagos_baz.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,SUCURSAL,FOLIO,RECUPERACION DE CAPITAL,RECUPERACION DE MORATORIOS,SALDO ACTUAL,MORATORIOS ACTUAL,FECHA GESTION,CARGO AUTOMATICO,cartera,Cartera
0,9701917,7,5,1-39-5608-13595,MARTHA ROSIO GARZA SALAZAR,8116124921,celular,2021-01-22 14:10:38,ESROSALESJA,ESPAÃA ROSALES JANET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
promesas_gestor_rec = pd.pivot_table(consolidado_pagos_baz,index=['usuario','nombre_usuario'],values=['monto_promesa','RECUPERACION DE CAPITAL','RECUPERACION DE MORATORIOS'],aggfunc=['count',np.sum])
promesas_gestor_rec = to_dataframe(promesas_gestor_rec)
promesas_gestor_rec = promesas_gestor_rec.iloc[:,[0,1,4,5,6,7]]
promesas_gestor_rec.columns = ['Usuario','Gestor','NumeroPromesas','SumaRecCapital','SumaRecMoratorios','SumaPromesas']
promesas_gestor_rec['Recuperado'] = promesas_gestor_rec['SumaRecCapital']+promesas_gestor_rec['SumaRecMoratorios']
promesas_gestor_rec

,Usuario,Gestor,NumeroPromesas,SumaRecCapital,SumaRecMoratorios,SumaPromesas,Recuperado
0,ALHENANDEZA,ALVARADO HERNANDEZ ADRIANA,684,254725.0,79154.0,372469.0,333879.0
1,ARCHUNDIAMABEAL,ARCHUNDIA MACIAS BERNARDO ALEJANDRO,474,188747.0,45479.0,324824.0,234226.0
2,BEBAUTISTAJI,JIMENEZ BAUTISTA BERNARDO,1090,700380.0,62901.0,962054.0,763281.0
3,BENITEZFLIS,ISAMAR BENITEZ FLORES,100,18144.0,11480.0,136786.0,29624.0
4,CARDOSASEVEBRE,BRENDA EVELYN CARDOSA SEVEROÂ,743,402824.0,118948.0,539330.0,521772.0
5,CARMONAHEEL,CARMONA HERNANDEZ ELIZABETH,71,24432.0,19921.0,69744.0,44353.0
6,CASTROGAHG,HUGO GERARDO CASTRO GARCIA,375,146079.0,45894.0,339831.0,191973.0
7,CRHERNAYA,YAQUELIN CRUZ HERNANDEZ,389,236521.0,46704.0,360626.0,283225.0
8,CRSALINASDM,CRUZ SALINAS DEYSI MICHELLE,2,1264.0,0.0,4276.0,1264.0
9,DIONISIOGAAL,ALEJANDRA DIONISIO GALICIAÂ,388,195431.0,101422.0,239523.0,296853.0


In [26]:
control_baz_rec = pd.merge(gestion_gestor_rec,promesas_gestor_rec,how='left',on='Usuario')
control_baz_rec.fillna(0,inplace=True)
control_baz_rec['Usuario2'] = [x[:3] for x in control_baz_rec['Usuario']]
control_baz_rec = control_baz_rec.loc[control_baz_rec['Usuario2']!='m08']
control_baz_rec.drop('Usuario2',axis=1,inplace=True)
control_baz_rec.head(1)

,Usuario,NombreUsuario,NumeroGestiones,Gestor,NumeroPromesas,SumaRecCapital,SumaRecMoratorios,SumaPromesas,Recuperado
0,AGUILARRIAL,AGUILAR RIOS DANIEL ALEJANDRO,96,0,0.0,0.0,0.0,0.0,0.0


In [30]:
final_baz_rec = pd.merge(control_baz_rec,control_performance_baz,how='left',on='Usuario')
final_baz_rec.fillna(0,inplace=True)
final_baz_rec['PromedioLlamadasHora'] = final_baz_rec['SumLlamadasDia']/final_baz_rec['SumHorasTrabajadas']
final_baz_rec['PromedioOcupacion'] = final_baz_rec['PromedioOcupacion']/100
final_baz_rec['EficienciaPagos'] = final_baz_rec['Recuperado']/final_baz_rec['SumaPromesas']
final_baz_rec['PromesasHora'] = final_baz_rec['NumeroPromesas']/final_baz_rec['SumHorasTrabajadas']
final_baz_rec['Usuario'] = final_baz_rec['Usuario'].str.upper()
final_baz_rec = pd.merge(final_baz_rec,personal,on='Usuario',how='left')
final_baz_rec['Estatus'].dropna(inplace=True)
final_baz_rec = final_baz_rec.loc[final_baz_rec['Estatus']=='COBRANZA']

In [31]:
final_baz_rec2 = final_baz_rec.iloc[:,[1,8,14,12,13,15]]
cols = ['Recuperado','EficienciaPagos','PromedioOcupacion','PromedioLlamadasHora','PromesasHora']
final_baz_rec2[cols] = final_baz_rec2[cols].round(2)
final_baz_rec2.fillna(0,inplace=True)
final_baz_rec2.head(1)

,NombreUsuario,Recuperado,EficienciaPagos,PromedioOcupacion,PromedioLlamadasHora,PromesasHora
1,ALVARADO HERNANDEZ ADRIANA,333879.0,0.9,0.29,30.13,3.29


In [29]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/Banco Azteca/Reporte Gestor Banco Azteca '+now+'.xlsx',engine='xlsxwriter')
final_baz_rec2.to_excel(writer,'BSC',index=False,header=True)
control_performance_baz.to_excel(writer,'PerformanceDetallado',index=False,header=True)
control_performance_baz_turno.to_excel(writer,'PerformanceTurno',index=False,header=True)
final_baz_rec.to_excel(writer,'GestionesDetallado',index=False,header=True)

writer.save()